## Demonstrates the use of craigsail_search Boats() class.

To do your own search, add your search category as show below. Gather raw data and use pandas to clean / process the data. 
Once you have figured out how to process the fields in your data, combine the functionality in a def and add it to craigsail_search by subclassing the Search() class. If you have any generic functions that can apply to any data, add them to the Search class.

CraigslistForSale.show_filters(category='boo')
Get category codes from craigslist.com by clicking on the desired sale category and looking in the url address.

Example: https://gainesville.craigslist.org/d/boats/search/boo - use the 'boo' at the end of the url

TODO: for 1k page limit / day, simply break cities up into separate calls using VPN switch

In [ ]:
import sys
import pandas as pd 
import numpy as np 
import geopandas as gpd 
from craigslist import CraigslistForSale
sys.path.append('../')
from craigsail_search import Boats, Bikes

pd.set_option('max_columns', 100)

: 

#### Set up your search paramaters 
Get more filters by looking at API documentation linked to above, or do a simple seach on a city and see which fields are returned / available for paramaters

## Sailboats

In [3]:
filters = {'query':'Sailboat', # set your boat query here
            'search_titles':True,
            # 'has_image':True, # not necessary if you're just tracking elements for price movements and inventory quantity
            'bundle_duplicates':True,
            #'boat_propulsion_type':'sail'
          }


port_cities = ['dallas','galveston','houston', # TX
               'seattle','olympic','skagit', # WA
                'micronesia','houma','caribbean','puertorico','virgin','mobile', # INTL
                'losangeles','monterey','sandiego','sfbay','oregoncoast', # CA
                'washingtondc', # D.C.
               'daytona','keys','miami','fortmyers','jacksonville','miami','panamacity','pensacola','sarasota','staugustine','tampa', # FL
                'jerseyshore','newyork', # NJ/NY
               'outerbanks', # NC
                'portland', # OR
               'hiltonhead','myrtlebeach','charleston', # SC
                'neworleans', # LA
                'brunswick','savannah','statesboro'] # GA

nearby_cities = [ 'asheville','eastnc', 'boone','hickory','greensboro','raleigh','fayetteville', 'wilmington', 'tricities', 'winstonsalem', 'onslow', # NC
                 'columbia', 'florence','greenville', # SC
                 'atlanta', 'augusta', 'athens','macon', 'albany', 'valdosta', 'nwga', # GA
                 'tallahassee', 'lakecity', 'gainesville', 'ocala', # FL
                'chattanooga', 'knoxville', # TN
                ] 


all_cities = port_cities + nearby_cities

#### Instantiate your search object and perform the search by calling the prep method
Depending on the number of cities, download could take several minutes

In [31]:
# replace with your local machine path
data_path = '/Users/destin/Documents/LOCALHOST/DEVELOPMENT/CraigSail/data' 

In [4]:
craigslist_sailboats = Boats('boo', data_path, *all_cities, **filters)

In [5]:
download_time, sailboats_df = craigslist_sailboats.prep_daily_sailboats_data()

../craigsail_search.py:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  years = df['name'].str.strip().str.extract(r'(?P<Year>\s{0,1}[1-2]\d{2,3}\s*)')
../craigsail_search.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # df[col] = pd.to_datetime(df[col]).dt.year
../craigsail_search.py:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [20]:
f'Download took: {round(download_time.seconds/60, 2)} minutes. {sailboats_df.index.size} records gathered.'

'Download took: 7.27 minutes. 746 records gathered.'

In [30]:
# an example of the data that is returned 
sailboats_df.head(2)

,id,repost_of,name,url,datetime,last_updated_2020-04-05,price_2020-04-05,where,has_image,geotag,body,created,images,condition,boat_propulsion_type,city,length overall (LOA),model name / number,propulsion type,make / manufacturer,year manufactured,engine hours (total),longitud total
0,7103346948,6636125352,Beautiful Mahogany Sunfish Sailboat,https://dallas.craigslist.org/dal/boa/d/lewisv...,2020-04-04 09:50:00,2020-04-04 09:50:00,2500.0,"Lewisville, TX",True,"(33.069644, -97.01752)",Classic Christmas Special\n\nBeautiful Classic...,2020-04-04 09:46:00,[],good,sail,dallas,13.0,Sunfish,sail,NaN,NaN,NaN,NaN
1,7101311774,6873639086,AMF Apollo Sailboat,https://dallas.craigslist.org/ndf/boa/d/wylie-...,2020-03-30 20:25:00,2020-03-30 20:25:00,900.0,None,True,"(33.0041, -96.5394)",I am helping my dad sell his 1980 AMF Apollo t...,2020-03-30 20:21:00,[],NaN,sail,dallas,16.0,NaN,sail,AMF Apollo,1980,NaN,NaN


In [26]:
# characteristics of the data returned
sailboats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       746 non-null    int64         
 1   repost_of                353 non-null    object        
 2   name                     746 non-null    object        
 3   url                      746 non-null    object        
 4   datetime                 746 non-null    datetime64[ns]
 5   last_updated_2020-04-05  746 non-null    datetime64[ns]
 6   price_2020-04-05         746 non-null    float64       
 7   where                    680 non-null    object        
 8   has_image                746 non-null    bool          
 9   geotag                   733 non-null    object        
 10  body                     746 non-null    object        
 11  created                  745 non-null    datetime64[ns]
 12  images                   746 non-nul

#### Clean the data
Note: If a class already exists for your search category and has cleaning methods, call them here. 
Otherwise, begin building the functionality. 

In [15]:
daily_cleaned = craigslist_sailboats.clean_city_sailboats_data(sailboats_df)

AttributeError: Can only use .str accessor with string values!

In [ ]:
strip_cols = ['repost_of', 'name', 'url', 'where', 'body', 'condition', 'boat_propulsion_type', 'make / manufacturer', 'propulsion type', 'model name / number']
daily_cleaned[strip_cols] = craigslist_sailboats.clean_str_columns(daily_cleaned[strip_cols])

In [65]:
daily_cleaned = craigslist_sailboats.strip_nan_columns(daily_cleaned)

#### Save the data
Go low-tech with .CSV files. More stable and easily replaced.

In [21]:
# suffix for unique file name
today = pd.to_datetime('today').strftime('%Y-%m-%d')

In [23]:
# rename columns for merging
daily_df = daily_df.rename({'last_updated': 'last_updated_' + today, 'price': 'price_' + today}, axis=1)

In [28]:
daily_df.to_csv(f'../data/all_boats_data {today}.csv', index=False)

## Bikes

In [9]:
filters = {'query':'touring', # set your boat query here
            'search_titles':True,
            # 'has_image':True, # not necessary if you're just tracking elements for price movements and inventory quantity
            'bundle_duplicates':True,
            #'boat_propulsion_type':'sail'
          }

nearby_cities = [ 'asheville','eastnc', 'boone','hickory','greensboro','raleigh','fayetteville', 'wilmington', 'tricities', 'winstonsalem', 'onslow', # NC
                 'columbia', 'florence','greenville', 'hiltonhead','myrtlebeach','charleston',# SC
                 'atlanta', 'augusta', 'athens','macon', 'albany', 'valdosta', 'nwga', 'brunswick','savannah','statesboro', # GA
                 'tallahassee', 'lakecity', 'gainesville', 'ocala', # FL
                'chattanooga', 'knoxville', # TN
                ]

#### Instantiate your search object and perform the search by calling the prep method
Depending on the number of cities, download could take several minutes

In [10]:
# replace with your local machine path
data_path = '/Users/destin/Documents/LOCALHOST/DEVELOPMENT/CraigSail/data' 

In [11]:
craigslist_bikes = Bikes('bia', data_path, *nearby_cities, **filters)

In [ ]:
download_time, bikes_df = craigslist_bikes.get_all_daily_postings()

In [ ]:
f'Download took: {round(download_time.seconds/60, 2)} minutes. {bikes_df.index.size} records gathered.'

In [ ]:
# an example of the data that is returned 
bikes_df.head(2)

In [ ]:
# characteristics of the data returned
bikes_df.info()

#### Clean the data
Note: If a class already exists for your search category and has cleaning methods, call them here. 
Otherwise, begin building the functionality. 

In [15]:
daily_cleaned = craigslist_bikes.clean_city_sailboats_data(bikes_df)

AttributeError: Can only use .str accessor with string values!

In [ ]:
strip_cols = ['repost_of', 'name', 'url', 'where', 'body', 'condition', 'boat_propulsion_type', 'make / manufacturer', 'propulsion type', 'model name / number']
daily_cleaned[strip_cols] = craigslist_sailboats.clean_str_columns(daily_cleaned[strip_cols])

In [65]:
daily_cleaned = craigslist_sailboats.strip_nan_columns(daily_cleaned)

#### Save the data
Go low-tech with .CSV files. More stable and easily replaced.

In [21]:
# suffix for unique file name
today = pd.to_datetime('today').strftime('%Y-%m-%d')

In [23]:
# rename columns for merging
daily_df = daily_df.rename({'last_updated': 'last_updated_' + today, 'price': 'price_' + today}, axis=1)

In [28]:
daily_df.to_csv(f'../data/all_boats_data {today}.csv', index=False)